# 模型训练

In [6]:
import pickle, sys, gc
import numpy as np
import pandas as pd
sys.path.append("./common")
sys.path.append("./dongdong/utils")
# model_save_dir = "/home/kesci/work/common/models/"
from utils import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TEST_NAMES, \
AnalysisCSV, ReduceMemUsage, ConcatFeatures, AnalysisHDF
OFFLINE = True
feature_base_dir = "/home/kesci/work/common/features/"
train_name = ORI_TRAIN_NAMES
if OFFLINE:
    train_file = "/home/kesci/work/inputs/train_flast_400w.csv"
    test_file = "/home/kesci/work/inputs/train_flast_100w.csv"
    in_train_feature_name = feature_base_dir + "nd_debugTrainConcatFeaturesV1"
    in_test_feature_name = feature_base_dir + "nd_debugTestConcatFeaturesV1"
    in_train_label = "/home/kesci/work/common/util_train/train_label_flast_400w.npy"
    in_test_label = "/home/kesci/work/common/util_train/train_label_flast_100w.npy"
    in_train_qid = "/home/kesci/work/common/util_train/train_qid_flast_400w.npy"
    in_test_qid = "/home/kesci/work/common/util_train/train_qid_flast_100w.npy"
    test_name = ORI_TRAIN_NAMES
else:
    train_file = "/home/kesci/work/inputs/train_first_5000w.csv"
    test_file = "/home/kesci/work/inputs/test_final_part1.csv"
    in_train_feature_name = feature_base_dir + "nd_onlineTrainConcatFeaturesV1"
    in_test_feature_name = feature_base_dir + "nd_onlineTestConcatFeaturesV1"
    in_train_label = "/home/kesci/work/common/util_train/train_label_flast_5000w.npy"
    in_test_label = None
    in_train_qid = "/home/kesci/work/common/util_train/train_qid_flast_5000w.npy"
    in_test_qid = "/home/kesci/work/common/util_train/test_qid.npy"
    test_name = ORI_TEST_NAMES
    
in_train_feature = in_train_feature_name + ".npy"
in_test_feature = in_test_feature_name + ".npy"
in_feature_names = in_train_feature_name + "_name.csv"

print("OFFLINE:", OFFLINE)
!ls -alh /home/kesci/work/common/features

OFFLINE: True
total 50G
drwxr-xr-x 7 kesci root  4.0K Aug  4 03:00 .
drwxr-xr-x 6 kesci root  4.0K Jul 31 16:21 ..
drwxr-xr-x 2 kesci root  4.0K Aug  5 07:59 balanceSamples
-rw-r--r-- 1 kesci users 2.1K Jul 28 08:40 dj_debug_concat_features_name.csv
-rw-r--r-- 1 kesci users 451M Jul 28 08:44 dj_debug_concat_features.npy
drwxr-xr-x 2 kesci root  4.0K Aug  4 06:02 magic
-rw-r--r-- 1 kesci users 2.1K Jul 29 23:54 nd_debugConcatFeatures_name.csv
-rw-r--r-- 1 kesci users 451M Jul 29 23:54 nd_debugConcatFeatures.npy
-rw-r--r-- 1 kesci users 367M Aug  7 14:24 nd_debugTestConcatFeaturesV1.npy
-rw-r--r-- 1 kesci users 2.0K Aug  7 14:24 nd_debugTrainConcatFeaturesV1_name.csv
-rw-r--r-- 1 kesci users 1.5G Aug  7 14:24 nd_debugTrainConcatFeaturesV1.npy
-rw-r--r-- 1 kesci users 7.2G Jul 30 16:10 nd_onlineTestConcatFeatures.npy
-rw-r--r-- 1 kesci users 6.3G Aug  4 03:09 nd_onlineTestConcatFeaturesV1.npy
-rw-r--r-- 1 kesci users 1.8K Jul 30 15:58 nd_onlineTrainConcatFeatures_name.csv
-rw-r--r-- 1 kes

In [10]:
! rm /home/kesci/work/common/features/nd_onlineTrainConcatFeaturesV1.npy

In [1]:
tmp = np.load("/home/kesci/work/common/features/balanceSamples/nd_onlineTrainConcatFeatures_2500W.npy")

In [2]:
print(tmp.shape)

(50633994, 48)


## 拼接特征

In [7]:
reconcat_features = True
if OFFLINE:
    # train_files = [
    #     "textMining/debug_trainTextMining_len.h5",
    #     "textMining/debug_trainTextMining_ngramSim.h5",
    #     "textMining/debug_trainTextMining_editSim.h5",
    #     # "textMining/debug_trainTextMining_hashSim.h5",
    #     "vectorSpace/debug_trainVectorSpace_tfidfVecLenV1.h5",
    #     "vectorSpace/debug_trainVectorSpace_tfidfVecSimV1.h5",
    #     "word2vec/debug_trainWord2vec_avgSenVecSim.h5",
    #     "magic/debug_trainMagic_localCTR.h5",
    #     "magic/debug_trainMagic_queryFreq.h5",
    #     "magic/debug_trainMagic_localTfidf.h5"
    # ]
    train_files = [
        "textMining/debug_trainTextMining_len.h5",
        "textMining/debug_trainTextMining_editSim.h5",
        "textMining/debug_trainTextMining_normalWordShare.h5",
        "textMining/debug_trainTextMining_idfWordShare.h5",
        "textMining/debug_trainTextMining_ngramSim.h5",
        "vectorSpace/debug_trainVectorSpace_tfidfVecFeat.h5",
        "word2vec/debug_trainWord2vec_avgSenVecSim.h5",
        "textMining/debug_trainTextMining_ngramDistance_flast_400w_xp.h5"
        # "textMining/debug_trainTextMining_ngramDistance_flast_400w_xp.h5"
        # "word2vec/debug_trainWord2vec_tfidfSenVecSim.h5",
        # "magic/debug_trainMagic_queryFreq.h5",
        # "magic/debug_trainMagic_localTfidf.h5"
    ]
    test_files = [
        "textMining/debug_testTextMining_len.h5",
        "textMining/debug_testTextMining_editSim.h5",
        "textMining/debug_testTextMining_normalWordShare.h5",
        "textMining/debug_testTextMining_idfWordShare.h5",
        "textMining/debug_testTextMining_ngramSim.h5",
        "vectorSpace/debug_testVectorSpace_tfidfVecFeat.h5",
        "word2vec/debug_testWord2vec_avgSenVecSim.h5",
        "textMining/debug_trainTextMining_ngramDistance_flast_100w_xp.h5"
        # "textMining/debug_trainTextMining_ngramDistance_xp.h5"
        # "word2vec/debug_testWord2vec_tfidfSenVecSim.h5",
        # "magic/debug_testMagic_queryFreq.h5",
        # "magic/debug_testMagic_localTfidf.h5"
    ]
else:
    train_files = [
        "textMining/online_trainTextMining_len.h5",
        "textMining/online_trainTextMining_editSim.h5",
        "word2vec/online_trainWord2vec_avgSenVecSim.h5",
        "vectorSpace/online_trainVectorSpace_tfidfVecFeat.h5",
        "textMining/online_trainTextMining_normalWordShare.h5",
        "textMining/online_trainTextMining_idfWordShare.h5"
    ]
    test_files = [
        "textMining/online_testTextMining_len.h5",
        "textMining/online_testTextMining_editSim.h5",
        "word2vec/online_testWord2vec_avgSenVecSim.h5",
        "vectorSpace/online_testVectorSpace_tfidfVecFeat.h5",
        "textMining/online_testTextMining_normalWordShare.h5",
        "textMining/online_testTextMining_idfWordShare.h5"
    ]
# 拼接成完整的路径
train_files = [feature_base_dir + _ for _ in train_files]
test_files = [feature_base_dir + _ for _ in test_files]

In [18]:
AnalysisHDF("/home/kesci/work/common/features/textMining/debug_trainTextMining_ngramDistance_flast_400w_xp.h5")

--->file shape:
(4000000, 20)
--->2 rows of file:
         0         1         2         3         4         5         6   \
0  0.809402  0.843590  0.753846  0.830769  0.218629  0.258352  0.147564   
1  0.819697  0.840909  0.781818  0.836364  0.241458  0.293173  0.168978   

         7    8    9    10   11        12   13   14   15        16   17   18  \
0  0.249970  1.0  1.0  1.0  1.0  0.166667  0.5  0.0  0.0  0.800000  0.8  0.8   
1  0.262222  1.0  1.0  1.0  1.0  0.166667  0.5  0.0  0.0  0.866667  1.0  0.8   

    19  
0  0.8  
1  0.8  


In [8]:
if reconcat_features:
    ConcatFeatures(train_files, in_train_feature_name)
    ConcatFeatures(test_files, in_test_feature_name, save_feature_name=False)

----->started 'load features' block...
Mem. usage decreased to 164.03 Mb (71.7% reduction)
Mem. usage decreased to 83.92 Mb (75.0% reduction)
Mem. usage decreased to 38.15 Mb (37.5% reduction)
Mem. usage decreased to  9.54 Mb (37.5% reduction)
Mem. usage decreased to 91.55 Mb (66.7% reduction)
Mem. usage decreased to 83.92 Mb (65.6% reduction)
Mem. usage decreased to 53.41 Mb (56.2% reduction)
Mem. usage decreased to 183.11 Mb (71.4% reduction)
----->finished 'load features' block, time used:101.41s.
----->started 'deal with invalid values' block...
----->finished 'deal with invalid values' block, time used:23.71s.
features shape: (4000000, 68)
two rows of features:
   debug_trainTextMining_text_qLen  debug_trainTextMining_text_tLen  \
0                               14                               62   
1                               14                               57   

   debug_trainTextMining_text_qtLenRatio  \
0                               0.225830   
1                      

In [4]:
if_rm_cols = False
rm_cols = ['debug_trainVectorSpace_tfidfVecFeat_paired_cosine_distances', 
'debug_trainVectorSpace_tfidfVecFeat_paired_euclidean_distances', 
'debug_trainVectorSpace_tfidfVecFeat_paired_manhattan_distances']
def remove_cols(load_npy, names_csv, rm_cols, save_npy):
    cols = list(pd.read_csv(names_csv, sep=",").columns.values)
    tmp = np.load(load_npy)
    rm_indexs = [i for i, _ in enumerate(cols) if _ in rm_cols]
    tmp = np.delete(tmp, rm_indexs, axis=1)
    np.save(save_npy, tmp)
    print("File saved to {}".format(save_npy))
    del tmp
    gc.collect()
if if_rm_cols:
    remove_cols(in_train_feature, in_feature_names, rm_cols, in_train_feature)
    remove_cols(in_test_feature, in_feature_names, rm_cols, in_test_feature)

File saved to /home/kesci/work/common/features/nd_debugTrainConcatFeaturesV1.npy
File saved to /home/kesci/work/common/features/nd_debugTestConcatFeaturesV1.npy


In [2]:
# tmp1 = np.load("/home/kesci/work/common/features/nd_debugTestConcatFeaturesV1.npy")
# tmp2 = np.load("/home/kesci/work/common/features/nd_debugTestConcatFeaturesV1.npy")
# print(tmp1[:10])
# print(tmp2[:10])

## 准备工作

提取label

In [2]:
def ExtractLabel(source_file, save_file):
    with Timer("extract label"):
        df = pd.read_csv(source_file, names=ORI_TRAIN_NAMES)
        np.save(save_file, df["label"].values)
        print("Label saved to {}".format(save_file))
        del df
        gc.collect()
# ExtractLabel(train_file, in_train_label)
# ExtractLabel(test_file, in_test_label)

----->started 'extract label' block...
Label saved to /home/kesci/work/common/util_train/train_label_flast_400w.npy
----->finished 'extract label' block, time used:6.49s.
----->started 'extract label' block...
Label saved to /home/kesci/work/common/util_train/train_label_flast_100w.npy
----->finished 'extract label' block, time used:2.0s.


In [3]:
tmp1 = np.load(in_train_label)
tmp2 = np.load(in_test_label)
print(tmp1[:10], tmp1.shape)
print(tmp2[:10], tmp2.shape)

[0 0 1 0 0 0 0 0 0 0] (4000000,)
[0 0 0 0 0 1 0 0 0 0] (1000000,)


提取query_id

In [4]:
def ExtractQid(source_file, save_file, names):
    with Timer("extract qid"):
        df = pd.read_csv(source_file, names=names)
        np.save(save_file, df["query_id"].values)
        print("Qid saved to {}".format(save_file))
        del df
        gc.collect()
ExtractQid(train_file, in_train_qid, names=ORI_TRAIN_NAMES)
if OFFLINE:
    ExtractQid(test_file, in_test_qid, names=ORI_TRAIN_NAMES)
else:
    ExtractQid(test_file, in_test_qid, names=ORI_TEST_NAMES)

----->started 'extract qid' block...
Qid saved to /home/kesci/work/common/util_train/train_qid_flast_400w.npy
----->finished 'extract qid' block, time used:6.03s.
----->started 'extract qid' block...
Qid saved to /home/kesci/work/common/util_train/train_qid_flast_100w.npy
----->finished 'extract qid' block, time used:1.52s.


In [21]:
tmp1 = np.load(in_train_qid)
tmp2 = np.load(in_test_qid)
print(tmp1[:10], tmp1.shape)
print(tmp2[:10], tmp2.shape)

[4847148 4847148 4847148 4847149 4847149 4847149 4847149 4847149 4847149
 4847149] (1000000,)
[4946681 4946681 4946681 4946681 4946681 4946681 4946681 4946681 4946682
 4946682] (400000,)


## 加载数据

In [9]:
from run_hyperopt1 import LoadDataset, run_lgb_gbdt_fast, run_lgb_gbdt
in_test_feature = in_test_feature_name + ".npy"
in_feature_names = in_train_feature_name + "_name.csv"
# 测试均衡数据集
# in_train_feature = "/home/kesci/work/common/features/balanceSamples/nd_onlineTrainConcatFeatures_2500W.npy"
# in_train_label = "/home/kesci/work/common/features/balanceSamples/train_label_last_2500W.npy"
# in_train_qid = None
if not OFFLINE: in_test_qid = None
# 加载数据集
LoadDataset(in_train_feature=in_train_feature, 
            in_train_label=in_train_label, 
            in_feature_names=in_feature_names,
            in_train_qid=in_train_qid,
            in_test_qid=in_test_qid,
            in_test_feature=in_test_feature,
            in_test_label=in_test_label)

{
 "in_train_feature": "/home/kesci/work/common/features/nd_debugTrainConcatFeaturesV1.npy",
 "in_train_label": "/home/kesci/work/common/util_train/train_label_flast_400w.npy",
 "in_feature_names": "/home/kesci/work/common/features/nd_debugTrainConcatFeaturesV1_name.csv",
 "in_train_qid": "/home/kesci/work/common/util_train/train_qid_flast_400w.npy",
 "in_test_qid": "/home/kesci/work/common/util_train/train_qid_flast_100w.npy",
 "in_test_feature": "/home/kesci/work/common/features/nd_debugTestConcatFeaturesV1.npy",
 "in_test_label": "/home/kesci/work/common/util_train/train_label_flast_100w.npy"
}
X shape: (4000000, 68)
X (2 rows):
[[14.         62.          0.22583008  4.4296875  62.         14.
  48.         38.          0.22583008  3.         13.          0.23071289
   4.33203125 13.          3.         10.          8.          0.23071289
  49.          0.50830078  0.50830078 34.         64.         37.
  71.         88.          6.          0.4284668   0.25        0.29003906
   0.2

In [7]:
# # print(tmp[-100:])
# count1 = 0
# count0 = 0
# count_other = 0
# for _ in tmp:
#     if _ == 0:
#         count0 += 1
#     elif _ == 1:
#         count1 += 1
#     else:
#         count_other += 1
# print(count1, count0, count_other)

9114119 9114119 0


## 训练模型

In [24]:
import random
np.random.seed(0) # cpu vars
random.seed(0) # Python
if OFFLINE:
    run_lgb_gbdt(prefix="nd", valid_type="kfold", cal_qauc=True, eval_num=5, save_result=False)
else:
    run_lgb_gbdt(prefix="nd", valid_type="simple", cal_qauc=False, eval_num=5, save_result=True)

{                                                  
 "bagging_fraction": 0.9,
 "bagging_freq": 6.0,
 "feature_fraction": 0.8,
 "lambda_l1": 0.5,
 "lambda_l2": 2.0,
 "learning_rate": 0.16,
 "max_bin": 255,
 "min_data_in_leaf": 40.0,
 "min_gain_to_split": 0.1,
 "num_iterations": 120.0,
 "num_leaves": 30.0,
 "num_threads": 5
}
[50]	valid's binary_logloss: 0.458268              
[100]	valid's binary_logloss: 0.457634             
[50]	valid's binary_logloss: 0.45817               
[100]	valid's binary_logloss: 0.457503             
[50]	valid's binary_logloss: 0.458481              
[100]	valid's binary_logloss: 0.457751             
[50]	valid's binary_logloss: 0.458327              
[100]	valid's binary_logloss: 0.457656             
[50]	valid's binary_logloss: 0.458134              
[100]	valid's binary_logloss: 0.457419             
混淆矩阵：                                              
[[3277657     946]                                 
 [ 719506    1891]]
feature importance:           

## 杂项

In [5]:
import os
print(os.getpid())

468


In [ ]:
! kill -9 468